# Repsly trial data

In [1]:
from repsly_data import RepslyData

repsly_data = RepslyData()
print('Reading data (this might take a minute or so)...', end='')
repsly_data.read_data('data/trial_users_analysis.csv', mode='FC')
print('done.')

Reading data (this might take a minute or so)...done.



Let's see what the data looks like:

In [2]:
read_batch = repsly_data.read_batch(batch_size=20)

X, y = next(read_batch)
print('X{}: {}'.format(list(X.shape), X))
print('y:', y)

X[20, 241]: [[ 303.    1.    4. ...,    0.    0.    0.]
 [ 192.    4.    3. ...,    0.    0.    0.]
 [ 363.    0.    0. ...,    0.    0.    0.]
 ..., 
 [ 180.    0.    0. ...,    0.    0.    0.]
 [ 336.    0.    0. ...,    0.    0.    0.]
 [ 459.    2.    3. ...,    0.    0.    0.]]
y: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]


As you can see above, each input vector `X` has `1+15*16=241` values, most of which are zeros. The first one is the trial start date as offset from `2016-01-01` and the rest is different usage parameters for the following `16` days. Data provided by batch read is randomly shuffled. Output values are stored in `y` and they represent if the user purchased the Repsly service after the trial or not.

# Training

First, we create a network with two fully connected hidden layers of size 250 and 50% dropout:

In [3]:
from repsly_nn import RepslyFC

repsly_nn = RepslyFC()

Then we train it for some number of epochs. (NB. there is s bug with restoring checkpoint)

In [4]:
arch = [250, 250]
keep_probs = [0.4, 0.5, 0.6]
input_keep_prob = 0.8
learning_rate = 0.001
decay_steps=10
decay_rate=0.99

batch_size = 1024
epochs_loops = 1
epochs_at_once = 40
skip_steps=10

stats = {}
for keep_prob in keep_probs:
    hyperparams = keep_prob
    arch_dict = {'keep_prob': keep_prob, 'input_keep_prob': input_keep_prob}
    
    repsly_nn.create_net(arch, arch_dict, learning_rate, decay_steps, decay_rate)
    for i in range(epochs_loops):
        print('Training for {} epochs...'.format(epochs_at_once))
        stats[hyperparams] = repsly_nn.train(data=repsly_data, batch_size=batch_size, epochs=epochs_at_once, skip_steps=skip_steps)
        print(stats[hyperparams])

Training for 40 epochs...
Checkpoint directory is: /Users/davor/PycharmProjects/deep_learning/repsly_challenge/checkpoints/RepslyFC-[250,250]/keep_prob-0.4/input_keep_prob-0.8/lr-0.001/dr-0.99/ds-10
Creating tf.train.Saver()...done
self.checkpoint_path: checkpoints/RepslyFC-[250,250]/keep_prob-0.4/input_keep_prob-0.8/lr-0.001/dr-0.99/ds-10
ckpt: None


TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a float into a Tensor.

In [ ]:
print({key: stats[key]['f1_score'] for key in stats.keys()})